In [241]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as pl
import os
import math

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV

from xgboost import XGBRegressor

from statsmodels.graphics.tsaplots import plot_acf

# Precios

In [242]:
cwd = os.getcwd()
data = pd.read_csv(cwd + '/datos_TFM_Celia.csv', sep = ',', engine='python')
data['Time'] = pd.to_datetime(data['Time'])

data = data[(data['Time'] < "2013-06-01 00:00:00") & (data['Time'] > "2013-04-30 23:00:00")]
df_precios = data[['Time', 'Power']]

data = pd.read_csv(cwd + '/Datos_Mercado' + '/PrecioDeCobroDesviosASubir.csv', sep = ';', engine='python')
df_precios['Desv_Subir'] = data['value'].values

data = pd.read_csv(cwd + '/Datos_Mercado' + '/PrecioDePagoDesviosABajar.csv', sep = ';', engine='python')
df_precios['Desv_Bajar'] = data['value'].values

df_precios['Diario'] = pd.Series(dtype='float')
df_precios['Continuo'] = pd.Series(dtype='float')
df_precios = df_precios.reset_index()
df_precios


,index,Time,Power,Desv_Subir,Desv_Bajar,Diario,Continuo
0,11663,2013-05-01 00:00:00,0.680481,7.53,78.25,NaN,NaN
1,11664,2013-05-01 01:00:00,0.591768,0.91,59.57,NaN,NaN
2,11665,2013-05-01 02:00:00,0.450803,65.17,65.17,NaN,NaN
3,11666,2013-05-01 03:00:00,0.374589,0.71,61.97,NaN,NaN
4,11667,2013-05-01 04:00:00,0.368011,0.01,17.83,NaN,NaN
...,...,...,...,...,...,...,...
739,12402,2013-05-31 19:00:00,0.000000,0.00,7.36,NaN,NaN
740,12403,2013-05-31 20:00:00,0.000000,0.88,56.82,NaN,NaN
741,12404,2013-05-31 21:00:00,0.000000,6.79,40.73,NaN,NaN
742,12405,2013-05-31 22:00:00,0.000000,1.22,60.52,NaN,NaN


## Continuo y Diario

In [ ]:
ruta_base = os.getcwd() + '/Datos_Mercado/'
nombre_base = 'INT_INTRA_C_MIN_MAX_1_'
extension = '.XLS'

for dia in range(1, 32):
    dia_str = f'{dia:02d}'
    nombre_archivo = f'{nombre_base}{dia_str}_05_2024_{dia_str}_05_2024{extension}'
    ruta_completa = ruta_base + nombre_archivo
    
    try:
        df = pd.read_excel(ruta_completa)
        df = df.reset_index(drop=True)
        df = df.iloc[3:]
        columna4 = df.iloc[:, 4] #cont
        columna5 = df.iloc[:, 5] #diario
        start_idx = (dia - 1) * 24
        for i in range(24):
            df_precios.at[start_idx + i, 'Continuo'] = columna4.iloc[i]
            df_precios.at[start_idx + i, 'Diario'] = columna5.iloc[i]
    
    
    except FileNotFoundError:
        print(f'El archivo {nombre_archivo} no se encontró.')

In [244]:
df_precios

,index,Time,Power,Desv_Subir,Desv_Bajar,Diario,Continuo
0,11663,2013-05-01 00:00:00,0.680481,7.53,78.25,35.01,28.81
1,11664,2013-05-01 01:00:00,0.591768,0.91,59.57,34.99,14.53
2,11665,2013-05-01 02:00:00,0.450803,65.17,65.17,21.07,24.84
3,11666,2013-05-01 03:00:00,0.374589,0.71,61.97,19.77,24.59
4,11667,2013-05-01 04:00:00,0.368011,0.01,17.83,17.83,13.50
...,...,...,...,...,...,...,...
739,12402,2013-05-31 19:00:00,0.000000,0.00,7.36,0.00,1.15
740,12403,2013-05-31 20:00:00,0.000000,0.88,56.82,3.25,5.21
741,12404,2013-05-31 21:00:00,0.000000,6.79,40.73,7.30,7.04
742,12405,2013-05-31 22:00:00,0.000000,1.22,60.52,5.50,5.07


# Predicciones

In [245]:
params_1 =  {'eta': 0.1, 'lambda': 10, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 300, 'subsample': 1.0}

def function(n_ahead):
    cwd = os.getcwd()
    data = pd.read_csv(cwd + '/datos_TFM_Celia.csv', sep = ',', engine='python')
    data['Time'] = pd.to_datetime(data['Time'])

    data.loc[:,'ws_lag1'] = data['ws'].shift(n_ahead)
    data.loc[:,'Power_lag1'] = data['Power'].shift(n_ahead)
    data.loc[:,'ws_lag2'] = data['ws'].shift(n_ahead+1)
    data.loc[:,'Power_lag2'] = data['Power'].shift(n_ahead+1)

    data.loc[:,'ws_lead1'] = data['ws'].shift(-1)
    data.loc[:,'ws_lead2'] = data['ws'].shift(-2)
    data = data.dropna()

    df_hist = data[data['Time'] < "2013-01-01 01:00:00"]
    df_fut = data[data['Time'] > "2013-01-01 00:00:00"]

    INPUTS = ['ws','wd', 'ws_lag1', 'ws_lag2', 'Power_lag1', 'Power_lag2','ws_lead1','ws_lead2']
    OUTPUT = ['Power']

    X = df_hist[INPUTS]
    y = df_hist[OUTPUT]

    X_train = X.iloc[0:round(0.8*X.shape[0])]
    X_test = X.iloc[round(0.8*X.shape[0])+1:X.shape[0]]
    y_train = y.iloc[0:round(0.8*X.shape[0])]
    y_test = y.iloc[round(0.8*X.shape[0])+1:X.shape[0]]


    X_fut = df_fut[INPUTS]
    y_fut = df_fut[OUTPUT]

    # Train the model with the best parameters
    best_xgb_model = XGBRegressor(
        booster='gbtree',
        objective='reg:squarederror',
        **params_1
    )

    best_xgb_model.fit(X_train, y_train)
    
    # Predict 
    y_pred = best_xgb_model.predict(X_fut)
    df_fut = df_fut.copy()
    df_fut.loc[:, 'XGB_pred'] = y_pred

    # Evaluate the model
    mae = mean_absolute_error(y_fut, y_pred)
    rmse = math.sqrt(mean_squared_error(y_fut, y_pred))
    
    booster = best_xgb_model.get_booster()
    trees = booster.get_dump(with_stats=True)
    num_params = sum(tree.count('\n') for tree in trees)
    print(f'Número de parámetros ajustados (nodos): {num_params}')

    return df_fut

## Continuo

In [246]:
horizonte = 2
df_fut = function(horizonte)
df_ev = df_fut[(df_fut['Time'] < "2013-06-01 00:00:00") & (df_fut['Time'] > "2013-04-30 23:00:00")]

df_ev = df_ev[['Time', 'Power', 'XGB_pred']]
df_ev.rename(columns={'XGB_pred': 'Pred_Continuo'}, inplace=True)
df_ev['Pred_Continuo'] = df_ev[['Pred_Continuo']].applymap(lambda x: x if x >= 0 else 0)

df_ev

Número de parámetros ajustados (nodos): 4438


/var/folders/ck/11mmsnfs1pgczh62kv6s4stw0000gn/T/ipykernel_91336/1111052058.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_ev['Pred_Continuo'] = df_ev[['Pred_Continuo']].applymap(lambda x: x if x >= 0 else 0)


,Time,Power,Pred_Continuo
11663,2013-05-01 00:00:00,0.680481,0.435806
11664,2013-05-01 01:00:00,0.591768,0.517627
11665,2013-05-01 02:00:00,0.450803,0.581979
11666,2013-05-01 03:00:00,0.374589,0.462799
11667,2013-05-01 04:00:00,0.368011,0.364440
...,...,...,...
12402,2013-05-31 19:00:00,0.000000,0.000000
12403,2013-05-31 20:00:00,0.000000,0.000000
12404,2013-05-31 21:00:00,0.000000,0.000000
12405,2013-05-31 22:00:00,0.000000,0.006059


## Diario

In [247]:
df_ev['Hour'] = df_ev['Time'].dt.hour
df_ev['Hour'] = df_ev['Hour'].replace(0, 24)
df_ev['Pred_Diario'] = pd.Series(dtype='float')

In [248]:
df_preds = df_ev.copy()

for i in range(1,25):    
    df_fut = function(i + 12)
    df_diario = df_fut[(df_fut['Time'] < "2013-06-01 00:00:00") & (df_fut['Time'] > "2013-04-30 23:00:00")]
    df_diario = df_diario[['Time', 'Power', 'XGB_pred']]
    df_preds.loc[df_preds['Hour'] == i, 'Pred_Diario'] = df_diario.loc[df_preds['Hour'] == i, 'XGB_pred'].values

Número de parámetros ajustados (nodos): 4452
Número de parámetros ajustados (nodos): 4478
Número de parámetros ajustados (nodos): 4468
Número de parámetros ajustados (nodos): 4492
Número de parámetros ajustados (nodos): 4490
Número de parámetros ajustados (nodos): 4482
Número de parámetros ajustados (nodos): 4466
Número de parámetros ajustados (nodos): 4478
Número de parámetros ajustados (nodos): 4454
Número de parámetros ajustados (nodos): 4492
Número de parámetros ajustados (nodos): 4484
Número de parámetros ajustados (nodos): 4444
Número de parámetros ajustados (nodos): 4478
Número de parámetros ajustados (nodos): 4482
Número de parámetros ajustados (nodos): 4486
Número de parámetros ajustados (nodos): 4482
Número de parámetros ajustados (nodos): 4500
Número de parámetros ajustados (nodos): 4496
Número de parámetros ajustados (nodos): 4486
Número de parámetros ajustados (nodos): 4462
Número de parámetros ajustados (nodos): 4460
Número de parámetros ajustados (nodos): 4478
Número de 

In [249]:
df_preds

,Time,Power,Pred_Continuo,Hour,Pred_Diario
11663,2013-05-01 00:00:00,0.680481,0.435806,24,0.628922
11664,2013-05-01 01:00:00,0.591768,0.517627,1,0.310871
11665,2013-05-01 02:00:00,0.450803,0.581979,2,0.349042
11666,2013-05-01 03:00:00,0.374589,0.462799,3,0.317264
11667,2013-05-01 04:00:00,0.368011,0.364440,4,0.266516
...,...,...,...,...,...
12402,2013-05-31 19:00:00,0.000000,0.000000,19,0.092521
12403,2013-05-31 20:00:00,0.000000,0.000000,20,0.071872
12404,2013-05-31 21:00:00,0.000000,0.000000,21,0.056267
12405,2013-05-31 22:00:00,0.000000,0.006059,22,0.044649


# Calculos con continuo

In [286]:
df_calc = df_preds.copy()
df_calc = df_calc[['Time','Power', 'Pred_Diario','Pred_Continuo']]
df_calc

,Time,Power,Pred_Diario,Pred_Continuo
11663,2013-05-01 00:00:00,0.680481,0.628922,0.435806
11664,2013-05-01 01:00:00,0.591768,0.310871,0.517627
11665,2013-05-01 02:00:00,0.450803,0.349042,0.581979
11666,2013-05-01 03:00:00,0.374589,0.317264,0.462799
11667,2013-05-01 04:00:00,0.368011,0.266516,0.364440
...,...,...,...,...
12402,2013-05-31 19:00:00,0.000000,0.092521,0.000000
12403,2013-05-31 20:00:00,0.000000,0.071872,0.000000
12404,2013-05-31 21:00:00,0.000000,0.056267,0.000000
12405,2013-05-31 22:00:00,0.000000,0.044649,0.006059


In [287]:
df_calc['Power'] = df_calc['Power']*45
df_calc['Pred_Diario'] = df_calc['Pred_Diario']*45
df_calc['Pred_Continuo'] = df_calc['Pred_Continuo']*45

In [288]:
df_calc['Compra_Cont'] = df_calc['Pred_Diario'] - df_calc['Pred_Continuo']
df_calc['Desvio_Subir'] = (df_calc['Power'] - df_calc['Pred_Continuo']).apply(lambda x: max(x, 0))
df_calc['Desvio_Bajar'] = (df_calc['Pred_Continuo'] - df_calc['Power']).apply(lambda x: max(x, 0))
df_calc = df_calc.reset_index()
df_calc

,index,Time,Power,Pred_Diario,Pred_Continuo,Compra_Cont,Desvio_Subir,Desvio_Bajar
0,11663,2013-05-01 00:00:00,30.621651,28.301495,19.611264,8.690231,11.010388,0.000000
1,11664,2013-05-01 01:00:00,26.629545,13.989213,23.293218,-9.304005,3.336328,0.000000
2,11665,2013-05-01 02:00:00,20.286157,15.706898,26.189041,-10.482143,0.000000,5.902884
3,11666,2013-05-01 03:00:00,16.856498,14.276901,20.825952,-6.549051,0.000000,3.969453
4,11667,2013-05-01 04:00:00,16.560473,11.993211,16.399784,-4.406573,0.160690,0.000000
...,...,...,...,...,...,...,...,...
739,12402,2013-05-31 19:00:00,0.000000,4.163447,0.000000,4.163447,0.000000,0.000000
740,12403,2013-05-31 20:00:00,0.000000,3.234221,0.000000,3.234221,0.000000,0.000000
741,12404,2013-05-31 21:00:00,0.000000,2.532007,0.000000,2.532007,0.000000,0.000000
742,12405,2013-05-31 22:00:00,0.000000,2.009188,0.272672,1.736516,0.000000,0.272672


In [289]:
df_precios

,index,Time,Power,Desv_Subir,Desv_Bajar,Diario,Continuo
0,11663,2013-05-01 00:00:00,0.680481,7.53,78.25,35.01,28.81
1,11664,2013-05-01 01:00:00,0.591768,0.91,59.57,34.99,14.53
2,11665,2013-05-01 02:00:00,0.450803,65.17,65.17,21.07,24.84
3,11666,2013-05-01 03:00:00,0.374589,0.71,61.97,19.77,24.59
4,11667,2013-05-01 04:00:00,0.368011,0.01,17.83,17.83,13.50
...,...,...,...,...,...,...,...
739,12402,2013-05-31 19:00:00,0.000000,0.00,7.36,0.00,1.15
740,12403,2013-05-31 20:00:00,0.000000,0.88,56.82,3.25,5.21
741,12404,2013-05-31 21:00:00,0.000000,6.79,40.73,7.30,7.04
742,12405,2013-05-31 22:00:00,0.000000,1.22,60.52,5.50,5.07


In [290]:
df_resultados = pd.DataFrame()
df_resultados['Venta_Diario'] = df_calc['Pred_Diario'] * df_precios['Diario']
df_resultados['Coste_Continuo'] = df_calc['Compra_Cont'] * df_precios['Continuo']
df_resultados['Desvio_Subir'] = df_calc['Desvio_Subir'] * df_precios['Desv_Subir']
df_resultados['Desvio_Bajar'] = df_calc['Desvio_Bajar'] * df_precios['Desv_Bajar']
df_resultados

,Venta_Diario,Coste_Continuo,Desvio_Subir,Desvio_Bajar
0,990.835330,250.365559,82.908221,0.000000
1,489.482551,-135.187193,3.036058,0.000000
2,330.944340,-260.376421,0.000000,384.690946
3,282.254333,-161.041152,0.000000,245.987031
4,213.838945,-59.488736,0.001607,0.000000
...,...,...,...,...
739,0.000000,4.787964,0.000000,0.000000
740,10.511217,16.850290,0.000000,0.000000
741,18.483651,17.825329,0.000000,0.000000
742,11.050536,8.804138,0.000000,16.502109


In [291]:
cobro_diario = df_resultados['Venta_Diario'].sum()
pago_cont = df_resultados['Coste_Continuo'].sum()
cobro_desv_subir = df_resultados['Desvio_Subir'].sum()
pago_desv_bajar = df_resultados['Desvio_Bajar'].sum()

print("Cobro Diario:", cobro_diario)
print("Pago Continuo:", pago_cont)
print("Cobro Desvio Subir:", cobro_desv_subir)
print("Pago Desvio Bajar:", pago_desv_bajar)

# Resultados sumando cobros y restando pagos
resultado_total = cobro_diario + cobro_desv_subir - pago_cont - pago_desv_bajar

print("Resultado:", resultado_total)


Cobro Diario: 253213.93831030373
Pago Continuo: 10662.279976235199
Cobro Desvio Subir: 12011.421202421492
Pago Desvio Bajar: 71376.30379011335
Resultado: 183186.77574637666


## Calculos sin continuo

In [295]:
df_calc_1 = df_preds.copy()
df_calc_1 = df_calc_1[['Time','Power', 'Pred_Diario']]
df_calc_1['Power'] = df_calc_1['Power']*45
df_calc_1['Pred_Diario'] = df_calc_1['Pred_Diario']*45
df_calc_1

,Time,Power,Pred_Diario
11663,2013-05-01 00:00:00,30.621651,28.301495
11664,2013-05-01 01:00:00,26.629545,13.989213
11665,2013-05-01 02:00:00,20.286157,15.706898
11666,2013-05-01 03:00:00,16.856498,14.276901
11667,2013-05-01 04:00:00,16.560473,11.993211
...,...,...,...
12402,2013-05-31 19:00:00,0.000000,4.163447
12403,2013-05-31 20:00:00,0.000000,3.234221
12404,2013-05-31 21:00:00,0.000000,2.532007
12405,2013-05-31 22:00:00,0.000000,2.009188


In [296]:
df_calc_1['Desvio_Subir'] = (df_calc_1['Power'] - df_calc_1['Pred_Diario']).apply(lambda x: max(x, 0))
df_calc_1['Desvio_Bajar'] = (df_calc_1['Pred_Diario'] - df_calc_1['Power']).apply(lambda x: max(x, 0))
df_calc_1 = df_calc_1.reset_index()
df_calc_1

,index,Time,Power,Pred_Diario,Desvio_Subir,Desvio_Bajar
0,11663,2013-05-01 00:00:00,30.621651,28.301495,2.320157,0.000000
1,11664,2013-05-01 01:00:00,26.629545,13.989213,12.640333,0.000000
2,11665,2013-05-01 02:00:00,20.286157,15.706898,4.579259,0.000000
3,11666,2013-05-01 03:00:00,16.856498,14.276901,2.579597,0.000000
4,11667,2013-05-01 04:00:00,16.560473,11.993211,4.567263,0.000000
...,...,...,...,...,...,...
739,12402,2013-05-31 19:00:00,0.000000,4.163447,0.000000,4.163447
740,12403,2013-05-31 20:00:00,0.000000,3.234221,0.000000,3.234221
741,12404,2013-05-31 21:00:00,0.000000,2.532007,0.000000,2.532007
742,12405,2013-05-31 22:00:00,0.000000,2.009188,0.000000,2.009188


In [298]:
df_resultados_1 = pd.DataFrame()
df_resultados_1['Venta_Diario'] = df_calc_1['Pred_Diario'] * df_precios['Diario']
df_resultados_1['Desvio_Subir'] = df_calc_1['Desvio_Subir'] * df_precios['Desv_Subir']
df_resultados_1['Desvio_Bajar'] = df_calc_1['Desvio_Bajar'] * df_precios['Desv_Bajar']

cobro_diario_1 = df_resultados_1['Venta_Diario'].sum()
cobro_desv_subir_1 = df_resultados_1['Desvio_Subir'].sum()
pago_desv_bajar_1 = df_resultados_1['Desvio_Bajar'].sum()

print("Cobro Diario:", cobro_diario_1)
print("Cobro Desvio Subir:", cobro_desv_subir_1)
print("Pago Desvio Bajar:", pago_desv_bajar_1)

# Resultados sumando cobros y restando pagos
resultado_total_1 = cobro_diario_1 + cobro_desv_subir_1 - pago_desv_bajar_1

print("Resultado:", resultado_total_1)

Cobro Diario: 253213.93831030373
Cobro Desvio Subir: 17611.57030666015
Pago Desvio Bajar: 113479.11745102966
Resultado: 157346.3911659342


In [299]:
resultado_total - resultado_total_1

25840.38458044245

In [278]:
df_calc.to_csv('predicciones.csv', index=False)
df_calc_1.to_csv('predicciones_sincont.csv', index=False)

df_precios.to_csv('precios.csv', index=False)
df_resultados.to_csv('resultados_cont.csv', index=False)
df_resultados_1.to_csv('resultados_sincont.csv', index=False)